# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [19]:
import pandas as pd
dest='LAX'
table='flights'
df=pd.read_csv('z_{}_{}.csv'.format(table,dest))
#remove columns based on data error
df=df.drop(['no_name'],axis=1)
#remove columns based on there will be no such columns in TEST data
#we keep 'arr_time' and 'cancelled' temporarily for future use
df=df.drop(['dep_time','dep_delay','taxi_out','wheels_on','wheels_off','taxi_in','cancellation_code'],axis=1)
df=df.drop(['diverted','actual_elapsed_time','air_time','carrier_delay','weather_delay','nas_delay','security_delay'],axis=1)
df=df.drop(['late_aircraft_delay','first_dep_time','total_add_gtime','longest_add_gtime'],axis=1)
#remove by common sense
df.groupby(['dest','dest_city_name']).size()
#dest_city_name is full depended on dest, so we can delete dest_city_name
df=df.drop(['dest_city_name'],axis=1)
#since we load data by per DEST, so we an delete dest as well
df=df.drop(['dest','dest_airport_id'],axis=1)
#dest_city_name is full depended on dest, so we can delete dest_city_name
df=df.drop(['origin_city_name'],axis=1)
df=df.drop(['origin_airport_id'],axis=1)
#remove by Data_description
df=df.drop(['branded_code_share','mkt_carrier'],axis=1)
#remove by value_counts
df=df.drop(['dup','flights'],axis=1)
#remove by assumption that fl_num doesn't have real impact on delay
df=df.drop(['mkt_carrier_fl_num','op_carrier_fl_num'],axis=1)
#remove by correlation on numerical columns;crs_elapsed_time is simillar with distance
df=df.drop(['distance'],axis=1)
#removed cancelled==1
df=df[df.cancelled!=1]
#There still have some records has no arr_delay, but they has crs_arr_time and arr_time, so we can calcluate
#df.arr_delay=np.where(df.arr_delay.isnull(),df.arr_time-df.crs_arr_time,df.arr_delay)
#don't try to calculate this kind of arr_delay, since it will generate more problem like -2297 delay, which is outlier
#just deleve them
df=df[df.arr_delay.notnull()]
#as you can see, all records has values now, so we don't need the arr_time and cancelled
df=df.drop(['arr_time','cancelled'],axis=1)
df['month']=pd.to_datetime(df['fl_date'],format='%Y-%m-%d').dt.month
df['day_of_month']=pd.to_datetime(df['fl_date'],format='%Y-%m-%d').dt.day
df['day_of_week']=pd.to_datetime(df['fl_date'],format='%Y-%m-%d').dt.dayofweek
df=df.drop(['fl_date'],axis=1)
import numpy as np
#df['arr_delay_log'] = np.log(df['arr_delay']+100) #sqrt, log2, log10 not behave better, so we just user log
#df['crs_elapsed_time_log']=np.log(df['crs_elapsed_time'])
#df=df.drop(['crs_elapsed_time'],axis=1)
#remove below tow features since they are too time consuming when doing regression
df=df.drop(['op_unique_carrier','tail_num'],axis=1)
df

C:\Users\zhouy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,mkt_unique_carrier,origin,crs_dep_time,crs_arr_time,arr_delay,crs_elapsed_time,month,day_of_month,day_of_week
0,DL,SEA,1704,1945,-11.0,161,11,27,2
1,DL,KOA,2311,619,-15.0,308,11,27,2
2,DL,AUS,1355,1513,-3.0,198,11,27,2
3,DL,ATL,940,1136,35.0,296,11,27,2
4,DL,SFO,1515,1645,38.0,90,11,27,2
...,...,...,...,...,...,...,...,...,...
500920,DL,LAS,1055,1204,2.0,69,11,27,2
500921,DL,SLC,2018,2120,24.0,122,11,27,2
500922,DL,HNL,2140,501,-15.0,321,11,27,2
500923,DL,ATL,1455,1651,-3.0,296,11,27,2


In [20]:
#For Skewed distributions: Use Inter-Quartile Range (IQR) proximity rule for turning it to a normal distrubution
#Interquartile range is given by, IQR = Q3 — Q1
Q3=df['arr_delay'].quantile(0.75)
Q1=df['arr_delay'].quantile(0.25)
IQR=Q3-Q1

upper_limit = Q3+1.5*IQR
print(upper_limit)
lower_limit = Q1-1.5*IQR
print(lower_limit)
#i am not sure to use this threshold to delete outlier or not
df_remove=df[(df.arr_delay<upper_limit) & (df.arr_delay>lower_limit)].copy()
df_remove.skew() #the crs_elapsed_time is down to 0.37 from 1.08

41.5
-50.5


crs_dep_time        0.095455
crs_arr_time       -0.175809
arr_delay           0.470978
crs_elapsed_time    0.367016
month              -0.028256
day_of_month        0.003046
day_of_week         0.035948
dtype: float64

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [21]:
df_remove['crs_dep_time_bin']=(df_remove['crs_dep_time']/100).apply(np.floor)
df_remove['crs_arr_time_bin']=(df_remove['crs_arr_time']/100).apply(np.floor)

In [22]:
df_remove

,mkt_unique_carrier,origin,crs_dep_time,crs_arr_time,arr_delay,crs_elapsed_time,month,day_of_month,day_of_week,crs_dep_time_bin,crs_arr_time_bin
0,DL,SEA,1704,1945,-11.0,161,11,27,2,17.0,19.0
1,DL,KOA,2311,619,-15.0,308,11,27,2,23.0,6.0
2,DL,AUS,1355,1513,-3.0,198,11,27,2,13.0,15.0
3,DL,ATL,940,1136,35.0,296,11,27,2,9.0,11.0
4,DL,SFO,1515,1645,38.0,90,11,27,2,15.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...
500920,DL,LAS,1055,1204,2.0,69,11,27,2,10.0,12.0
500921,DL,SLC,2018,2120,24.0,122,11,27,2,20.0,21.0
500922,DL,HNL,2140,501,-15.0,321,11,27,2,21.0,5.0
500923,DL,ATL,1455,1651,-3.0,296,11,27,2,14.0,16.0


In [23]:
df_remove=df_remove.drop(['crs_arr_time','crs_dep_time'],axis=1)
df_remove

,mkt_unique_carrier,origin,arr_delay,crs_elapsed_time,month,day_of_month,day_of_week,crs_dep_time_bin,crs_arr_time_bin
0,DL,SEA,-11.0,161,11,27,2,17.0,19.0
1,DL,KOA,-15.0,308,11,27,2,23.0,6.0
2,DL,AUS,-3.0,198,11,27,2,13.0,15.0
3,DL,ATL,35.0,296,11,27,2,9.0,11.0
4,DL,SFO,38.0,90,11,27,2,15.0,16.0
...,...,...,...,...,...,...,...,...,...
500920,DL,LAS,2.0,69,11,27,2,10.0,12.0
500921,DL,SLC,24.0,122,11,27,2,20.0,21.0
500922,DL,HNL,-15.0,321,11,27,2,21.0,5.0
500923,DL,ATL,-3.0,296,11,27,2,14.0,16.0


In [24]:
df_remove=pd.get_dummies(df_remove,columns=['mkt_unique_carrier','origin'])

In [25]:
df_remove.shape

(455215, 130)

In [26]:
df_remove.columns

Index(['arr_delay', 'crs_elapsed_time', 'month', 'day_of_month', 'day_of_week',
       'crs_dep_time_bin', 'crs_arr_time_bin', 'mkt_unique_carrier_AA',
       'mkt_unique_carrier_AS', 'mkt_unique_carrier_B6',
       ...
       'origin_SLC', 'origin_SMF', 'origin_STL', 'origin_STS', 'origin_SUN',
       'origin_TPA', 'origin_TUL', 'origin_TUS', 'origin_TWF', 'origin_XNA'],
      dtype='object', length=130)

In [27]:
df_remove

,arr_delay,crs_elapsed_time,month,day_of_month,day_of_week,crs_dep_time_bin,crs_arr_time_bin,mkt_unique_carrier_AA,mkt_unique_carrier_AS,mkt_unique_carrier_B6,...,origin_SLC,origin_SMF,origin_STL,origin_STS,origin_SUN,origin_TPA,origin_TUL,origin_TUS,origin_TWF,origin_XNA
0,-11.0,161,11,27,2,17.0,19.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-15.0,308,11,27,2,23.0,6.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-3.0,198,11,27,2,13.0,15.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,35.0,296,11,27,2,9.0,11.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,38.0,90,11,27,2,15.0,16.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500920,2.0,69,11,27,2,10.0,12.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
500921,24.0,122,11,27,2,20.0,21.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
500922,-15.0,321,11,27,2,21.0,5.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
500923,-3.0,296,11,27,2,14.0,16.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

In [32]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df_remove.drop(['arr_delay'],axis=1),
                                                    df_remove['arr_delay'],
                                                    test_size=0.3, random_state=42)

In [33]:
train_x.shape

(318650, 129)

In [37]:
#Random Forest
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state=13,n_estimators=100)
#10 tree get 0.16
rfr.fit(train_x, train_y)
y_pred = rfr.predict(test_x)


In [38]:
y_pred.shape

(136565,)

In [39]:
from sklearn.metrics import mean_squared_error,r2_score
print("MSE = {}".format(mean_squared_error(test_y,y_pred)))
print("R2 Score = {}".format(r2_score(test_y,y_pred)))

MSE = 221.57633795695483
R2 Score = 0.1658761078328278


In [40]:
test_y

429555   -16.0
239399   -12.0
99364     15.0
233299   -13.0
409941   -20.0
          ... 
333425    -1.0
214745    -5.0
83624    -16.0
453818    29.0
194306   -29.0
Name: arr_delay, Length: 136565, dtype: float64

In [41]:
y_pred

array([-14.11,  -6.82,   2.7 , ..., -15.41,  -3.85, -16.33])

In [43]:
#XGBoost
import xgboost as xgb
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.8, learning_rate = 0.1,max_depth = 7, 
                          alpha = 10, n_estimators = 1000)
#depth=5, n_estimator=5000 get 0.17
#depth=7, n_estimator=1000 get 0.17
#colsample_bytree=0.8, depth=7, n_estimator=1000 get 0.212
#colsample_bytree=0.8, depth=7, n_estimator=5000 get 0.176


In [44]:
xg_reg.fit(train_x,train_y)
y_pred = xg_reg.predict(test_x)

In [45]:
print("MSE = {}".format(mean_squared_error(test_y,y_pred)))
print("R2 Score = {}".format(r2_score(test_y,y_pred)))

MSE = 209.0804871804685
R2 Score = 0.2129167249931665


### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

In [97]:
#see what is the format the input should look like 
#df_remove.drop(['arr_delay','arr_delay_log'],axis=1)
train_x

,crs_elapsed_time,month,day_of_month,day_of_week,crs_dep_time_bin,crs_arr_time_bin,mkt_unique_carrier_AA,mkt_unique_carrier_AS,mkt_unique_carrier_B6,mkt_unique_carrier_DL,...,origin_SLC,origin_SMF,origin_STL,origin_STS,origin_SUN,origin_TPA,origin_TUL,origin_TUS,origin_TWF,origin_XNA
341228,320,10,6,6,20.0,5.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
498324,82,11,23,5,14.0,15.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
61725,140,12,26,3,19.0,21.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
361721,150,11,6,1,14.0,15.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116905,108,5,16,2,10.0,11.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284666,210,7,18,3,15.0,17.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
401983,153,3,4,6,21.0,0.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
144918,241,2,27,1,7.0,9.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
160848,150,9,22,5,12.0,14.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [98]:
train_x.columns

Index(['crs_elapsed_time', 'month', 'day_of_month', 'day_of_week',
       'crs_dep_time_bin', 'crs_arr_time_bin', 'mkt_unique_carrier_AA',
       'mkt_unique_carrier_AS', 'mkt_unique_carrier_B6',
       'mkt_unique_carrier_DL',
       ...
       'origin_SLC', 'origin_SMF', 'origin_STL', 'origin_STS', 'origin_SUN',
       'origin_TPA', 'origin_TUL', 'origin_TUS', 'origin_TWF', 'origin_XNA'],
      dtype='object', length=129)

In [111]:

df_sub=pd.read_csv('z_flights_test_{}.csv'.format(dest))
df_sub

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01 00:00:00,WN,WN,WN,1997,WN,N8507C,1997,14057,PDX,"Portland, OR",12892,LAX,"Los Angeles, CA",1835,2100,N,145,1,834
1,2020-01-01 00:00:00,WN,WN,WN,6117,WN,N453WN,6117,14057,PDX,"Portland, OR",12892,LAX,"Los Angeles, CA",605,830,N,145,1,834
2,2020-01-01 00:00:00,WN,WN,WN,1674,WN,N440LV,1674,14107,PHX,"Phoenix, AZ",12892,LAX,"Los Angeles, CA",2255,2315,N,80,1,370
3,2020-01-01 00:00:00,WN,WN,WN,3466,WN,N8571Z,3466,14107,PHX,"Phoenix, AZ",12892,LAX,"Los Angeles, CA",1705,1735,N,90,1,370
4,2020-01-01 00:00:00,WN,WN,WN,4657,WN,N7731A,4657,14107,PHX,"Phoenix, AZ",12892,LAX,"Los Angeles, CA",1335,1405,N,90,1,370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20074,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5981,OO,N788SK,5981,10372,ASE,"Aspen, CO",12892,LAX,"Los Angeles, CA",740,929,N,169,1,737
20075,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5989,OO,N905SW,5989,13476,MRY,"Monterey, CA",12892,LAX,"Los Angeles, CA",1331,1500,N,89,1,266
20076,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5994,OO,N167SY,5994,12441,JAC,"Jackson, WY",12892,LAX,"Los Angeles, CA",1927,2112,N,165,1,784
20077,2020-01-31 00:00:00,AA,AA_CODESHARE,AA,3239,OO,N703SK,3239,11638,FAT,"Fresno, CA",12892,LAX,"Los Angeles, CA",627,743,N,76,1,209


In [112]:
df_sub=df_sub.drop(['dest_city_name'],axis=1)
df_sub

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01 00:00:00,WN,WN,WN,1997,WN,N8507C,1997,14057,PDX,"Portland, OR",12892,LAX,1835,2100,N,145,1,834
1,2020-01-01 00:00:00,WN,WN,WN,6117,WN,N453WN,6117,14057,PDX,"Portland, OR",12892,LAX,605,830,N,145,1,834
2,2020-01-01 00:00:00,WN,WN,WN,1674,WN,N440LV,1674,14107,PHX,"Phoenix, AZ",12892,LAX,2255,2315,N,80,1,370
3,2020-01-01 00:00:00,WN,WN,WN,3466,WN,N8571Z,3466,14107,PHX,"Phoenix, AZ",12892,LAX,1705,1735,N,90,1,370
4,2020-01-01 00:00:00,WN,WN,WN,4657,WN,N7731A,4657,14107,PHX,"Phoenix, AZ",12892,LAX,1335,1405,N,90,1,370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20074,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5981,OO,N788SK,5981,10372,ASE,"Aspen, CO",12892,LAX,740,929,N,169,1,737
20075,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5989,OO,N905SW,5989,13476,MRY,"Monterey, CA",12892,LAX,1331,1500,N,89,1,266
20076,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5994,OO,N167SY,5994,12441,JAC,"Jackson, WY",12892,LAX,1927,2112,N,165,1,784
20077,2020-01-31 00:00:00,AA,AA_CODESHARE,AA,3239,OO,N703SK,3239,11638,FAT,"Fresno, CA",12892,LAX,627,743,N,76,1,209


In [113]:
df_sub=df_sub.drop(['dest','dest_airport_id'],axis=1)
df_sub

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01 00:00:00,WN,WN,WN,1997,WN,N8507C,1997,14057,PDX,"Portland, OR",1835,2100,N,145,1,834
1,2020-01-01 00:00:00,WN,WN,WN,6117,WN,N453WN,6117,14057,PDX,"Portland, OR",605,830,N,145,1,834
2,2020-01-01 00:00:00,WN,WN,WN,1674,WN,N440LV,1674,14107,PHX,"Phoenix, AZ",2255,2315,N,80,1,370
3,2020-01-01 00:00:00,WN,WN,WN,3466,WN,N8571Z,3466,14107,PHX,"Phoenix, AZ",1705,1735,N,90,1,370
4,2020-01-01 00:00:00,WN,WN,WN,4657,WN,N7731A,4657,14107,PHX,"Phoenix, AZ",1335,1405,N,90,1,370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20074,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5981,OO,N788SK,5981,10372,ASE,"Aspen, CO",740,929,N,169,1,737
20075,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5989,OO,N905SW,5989,13476,MRY,"Monterey, CA",1331,1500,N,89,1,266
20076,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5994,OO,N167SY,5994,12441,JAC,"Jackson, WY",1927,2112,N,165,1,784
20077,2020-01-31 00:00:00,AA,AA_CODESHARE,AA,3239,OO,N703SK,3239,11638,FAT,"Fresno, CA",627,743,N,76,1,209


In [114]:
df_sub=df_sub.drop(['origin_city_name'],axis=1)
df_sub=df_sub.drop(['origin_airport_id'],axis=1)
#remove by Data_description
df_sub=df_sub.drop(['branded_code_share','mkt_carrier'],axis=1)
#remove by value_counts
df_sub=df_sub.drop(['dup','flights'],axis=1)
df_sub.head(5)

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin,crs_dep_time,crs_arr_time,crs_elapsed_time,distance
0,2020-01-01 00:00:00,WN,1997,WN,N8507C,1997,PDX,1835,2100,145,834
1,2020-01-01 00:00:00,WN,6117,WN,N453WN,6117,PDX,605,830,145,834
2,2020-01-01 00:00:00,WN,1674,WN,N440LV,1674,PHX,2255,2315,80,370
3,2020-01-01 00:00:00,WN,3466,WN,N8571Z,3466,PHX,1705,1735,90,370
4,2020-01-01 00:00:00,WN,4657,WN,N7731A,4657,PHX,1335,1405,90,370


In [115]:
df_sub=df_sub.drop(['mkt_carrier_fl_num','op_carrier_fl_num'],axis=1)
df_sub.head(5)

,fl_date,mkt_unique_carrier,op_unique_carrier,tail_num,origin,crs_dep_time,crs_arr_time,crs_elapsed_time,distance
0,2020-01-01 00:00:00,WN,WN,N8507C,PDX,1835,2100,145,834
1,2020-01-01 00:00:00,WN,WN,N453WN,PDX,605,830,145,834
2,2020-01-01 00:00:00,WN,WN,N440LV,PHX,2255,2315,80,370
3,2020-01-01 00:00:00,WN,WN,N8571Z,PHX,1705,1735,90,370
4,2020-01-01 00:00:00,WN,WN,N7731A,PHX,1335,1405,90,370


In [116]:
df_sub=df_sub.drop(['op_unique_carrier','tail_num','distance'],axis=1)
df_sub.head(5)

,fl_date,mkt_unique_carrier,origin,crs_dep_time,crs_arr_time,crs_elapsed_time
0,2020-01-01 00:00:00,WN,PDX,1835,2100,145
1,2020-01-01 00:00:00,WN,PDX,605,830,145
2,2020-01-01 00:00:00,WN,PHX,2255,2315,80
3,2020-01-01 00:00:00,WN,PHX,1705,1735,90
4,2020-01-01 00:00:00,WN,PHX,1335,1405,90


In [117]:
df_sub.notnull().count()

fl_date               20079
mkt_unique_carrier    20079
origin                20079
crs_dep_time          20079
crs_arr_time          20079
crs_elapsed_time      20079
dtype: int64

In [118]:
df_sub

,fl_date,mkt_unique_carrier,origin,crs_dep_time,crs_arr_time,crs_elapsed_time
0,2020-01-01 00:00:00,WN,PDX,1835,2100,145
1,2020-01-01 00:00:00,WN,PDX,605,830,145
2,2020-01-01 00:00:00,WN,PHX,2255,2315,80
3,2020-01-01 00:00:00,WN,PHX,1705,1735,90
4,2020-01-01 00:00:00,WN,PHX,1335,1405,90
...,...,...,...,...,...,...
20074,2020-01-31 00:00:00,UA,ASE,740,929,169
20075,2020-01-31 00:00:00,UA,MRY,1331,1500,89
20076,2020-01-31 00:00:00,UA,JAC,1927,2112,165
20077,2020-01-31 00:00:00,AA,FAT,627,743,76


In [119]:
df_sub['month']=pd.to_datetime(df_sub['fl_date'],format='%Y-%m-%d').dt.month
df_sub['day_of_month']=pd.to_datetime(df_sub['fl_date'],format='%Y-%m-%d').dt.day
df_sub['day_of_week']=pd.to_datetime(df_sub['fl_date'],format='%Y-%m-%d').dt.dayofweek
df_sub=df_sub.drop(['fl_date'],axis=1)
df_sub

,mkt_unique_carrier,origin,crs_dep_time,crs_arr_time,crs_elapsed_time,month,day_of_month,day_of_week
0,WN,PDX,1835,2100,145,1,1,2
1,WN,PDX,605,830,145,1,1,2
2,WN,PHX,2255,2315,80,1,1,2
3,WN,PHX,1705,1735,90,1,1,2
4,WN,PHX,1335,1405,90,1,1,2
...,...,...,...,...,...,...,...,...
20074,UA,ASE,740,929,169,1,31,4
20075,UA,MRY,1331,1500,89,1,31,4
20076,UA,JAC,1927,2112,165,1,31,4
20077,AA,FAT,627,743,76,1,31,4


In [120]:
df_sub['crs_dep_time_bin']=(df_sub['crs_dep_time']/100).apply(np.floor)
df_sub['crs_arr_time_bin']=(df_sub['crs_arr_time']/100).apply(np.floor)
df_sub=df_sub.drop(['crs_arr_time','crs_dep_time'],axis=1)
df_sub=pd.get_dummies(df_sub,columns=['mkt_unique_carrier','origin'])
df_sub.columns

Index(['crs_elapsed_time', 'month', 'day_of_month', 'day_of_week',
       'crs_dep_time_bin', 'crs_arr_time_bin', 'mkt_unique_carrier_AA',
       'mkt_unique_carrier_AS', 'mkt_unique_carrier_B6',
       'mkt_unique_carrier_DL',
       ...
       'origin_SLC', 'origin_SMF', 'origin_STL', 'origin_STS', 'origin_SUN',
       'origin_TPA', 'origin_TUL', 'origin_TUS', 'origin_TWF', 'origin_XNA'],
      dtype='object', length=114)

In [121]:
#submission data has no new columns than test_x
set(df_sub.columns.to_list())-set(test_x.columns.to_list())

set()

In [122]:
#submission data has few columns than test_x
set(test_x.columns.to_list())-set(df_sub.columns.to_list())

{'mkt_unique_carrier_VX',
 'origin_BIL',
 'origin_CID',
 'origin_DRO',
 'origin_DSM',
 'origin_FAR',
 'origin_FLG',
 'origin_FSD',
 'origin_ICT',
 'origin_IDA',
 'origin_LIT',
 'origin_MFE',
 'origin_OGD',
 'origin_RAP',
 'origin_SBN'}

In [123]:
#Then we need to add tha above columns to df_sub
for val in set(test_x.columns.to_list())-set(df_sub.columns.to_list()):
    print(val)
    df_sub[val]=0

origin_CID
origin_ICT
origin_DRO
origin_LIT
origin_SBN
origin_FAR
origin_RAP
origin_BIL
origin_DSM
origin_OGD
origin_IDA
origin_MFE
origin_FLG
mkt_unique_carrier_VX
origin_FSD


In [124]:
df_sub.columns

Index(['crs_elapsed_time', 'month', 'day_of_month', 'day_of_week',
       'crs_dep_time_bin', 'crs_arr_time_bin', 'mkt_unique_carrier_AA',
       'mkt_unique_carrier_AS', 'mkt_unique_carrier_B6',
       'mkt_unique_carrier_DL',
       ...
       'origin_FAR', 'origin_RAP', 'origin_BIL', 'origin_DSM', 'origin_OGD',
       'origin_IDA', 'origin_MFE', 'origin_FLG', 'mkt_unique_carrier_VX',
       'origin_FSD'],
      dtype='object', length=129)

In [125]:
df_sub_final=df_sub[test_x.columns.to_list()]

In [127]:
df_sub_final

,crs_elapsed_time,month,day_of_month,day_of_week,crs_dep_time_bin,crs_arr_time_bin,mkt_unique_carrier_AA,mkt_unique_carrier_AS,mkt_unique_carrier_B6,mkt_unique_carrier_DL,...,origin_SLC,origin_SMF,origin_STL,origin_STS,origin_SUN,origin_TPA,origin_TUL,origin_TUS,origin_TWF,origin_XNA
0,145,1,1,2,18.0,21.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,145,1,1,2,6.0,8.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,80,1,1,2,22.0,23.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,90,1,1,2,17.0,17.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,90,1,1,2,13.0,14.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20074,169,1,31,4,7.0,9.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20075,89,1,31,4,13.0,15.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20076,165,1,31,4,19.0,21.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20077,76,1,31,4,6.0,7.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
df_sub_final.columns

Index(['crs_elapsed_time', 'month', 'day_of_month', 'day_of_week',
       'crs_dep_time_bin', 'crs_arr_time_bin', 'mkt_unique_carrier_AA',
       'mkt_unique_carrier_AS', 'mkt_unique_carrier_B6',
       'mkt_unique_carrier_DL',
       ...
       'origin_SLC', 'origin_SMF', 'origin_STL', 'origin_STS', 'origin_SUN',
       'origin_TPA', 'origin_TUL', 'origin_TUS', 'origin_TWF', 'origin_XNA'],
      dtype='object', length=129)

In [129]:
#make prediction
y_pred = xg_reg.predict(df_sub_final)

In [130]:
y_pred

array([-6.4541354, -8.531584 , 13.024975 , ..., -9.122553 , -5.616225 ,
       -8.777413 ], dtype=float32)

In [131]:
df_all=pd.read_csv('z_flights_test_{}.csv'.format(dest))
df_all

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01 00:00:00,WN,WN,WN,1997,WN,N8507C,1997,14057,PDX,"Portland, OR",12892,LAX,"Los Angeles, CA",1835,2100,N,145,1,834
1,2020-01-01 00:00:00,WN,WN,WN,6117,WN,N453WN,6117,14057,PDX,"Portland, OR",12892,LAX,"Los Angeles, CA",605,830,N,145,1,834
2,2020-01-01 00:00:00,WN,WN,WN,1674,WN,N440LV,1674,14107,PHX,"Phoenix, AZ",12892,LAX,"Los Angeles, CA",2255,2315,N,80,1,370
3,2020-01-01 00:00:00,WN,WN,WN,3466,WN,N8571Z,3466,14107,PHX,"Phoenix, AZ",12892,LAX,"Los Angeles, CA",1705,1735,N,90,1,370
4,2020-01-01 00:00:00,WN,WN,WN,4657,WN,N7731A,4657,14107,PHX,"Phoenix, AZ",12892,LAX,"Los Angeles, CA",1335,1405,N,90,1,370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20074,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5981,OO,N788SK,5981,10372,ASE,"Aspen, CO",12892,LAX,"Los Angeles, CA",740,929,N,169,1,737
20075,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5989,OO,N905SW,5989,13476,MRY,"Monterey, CA",12892,LAX,"Los Angeles, CA",1331,1500,N,89,1,266
20076,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5994,OO,N167SY,5994,12441,JAC,"Jackson, WY",12892,LAX,"Los Angeles, CA",1927,2112,N,165,1,784
20077,2020-01-31 00:00:00,AA,AA_CODESHARE,AA,3239,OO,N703SK,3239,11638,FAT,"Fresno, CA",12892,LAX,"Los Angeles, CA",627,743,N,76,1,209


In [132]:
df_all['arr_delay']=y_pred

In [133]:
df_all

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay
0,2020-01-01 00:00:00,WN,WN,WN,1997,WN,N8507C,1997,14057,PDX,...,12892,LAX,"Los Angeles, CA",1835,2100,N,145,1,834,-6.454135
1,2020-01-01 00:00:00,WN,WN,WN,6117,WN,N453WN,6117,14057,PDX,...,12892,LAX,"Los Angeles, CA",605,830,N,145,1,834,-8.531584
2,2020-01-01 00:00:00,WN,WN,WN,1674,WN,N440LV,1674,14107,PHX,...,12892,LAX,"Los Angeles, CA",2255,2315,N,80,1,370,13.024975
3,2020-01-01 00:00:00,WN,WN,WN,3466,WN,N8571Z,3466,14107,PHX,...,12892,LAX,"Los Angeles, CA",1705,1735,N,90,1,370,-0.513752
4,2020-01-01 00:00:00,WN,WN,WN,4657,WN,N7731A,4657,14107,PHX,...,12892,LAX,"Los Angeles, CA",1335,1405,N,90,1,370,-4.727969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20074,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5981,OO,N788SK,5981,10372,ASE,...,12892,LAX,"Los Angeles, CA",740,929,N,169,1,737,-2.018487
20075,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5989,OO,N905SW,5989,13476,MRY,...,12892,LAX,"Los Angeles, CA",1331,1500,N,89,1,266,2.137516
20076,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5994,OO,N167SY,5994,12441,JAC,...,12892,LAX,"Los Angeles, CA",1927,2112,N,165,1,784,-9.122553
20077,2020-01-31 00:00:00,AA,AA_CODESHARE,AA,3239,OO,N703SK,3239,11638,FAT,...,12892,LAX,"Los Angeles, CA",627,743,N,76,1,209,-5.616225


In [134]:
df_all.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance',
       'arr_delay'],
      dtype='object')

In [135]:
df_all['predicted_delay']=df_all['arr_delay']

In [136]:
df_all

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay,predicted_delay
0,2020-01-01 00:00:00,WN,WN,WN,1997,WN,N8507C,1997,14057,PDX,...,LAX,"Los Angeles, CA",1835,2100,N,145,1,834,-6.454135,-6.454135
1,2020-01-01 00:00:00,WN,WN,WN,6117,WN,N453WN,6117,14057,PDX,...,LAX,"Los Angeles, CA",605,830,N,145,1,834,-8.531584,-8.531584
2,2020-01-01 00:00:00,WN,WN,WN,1674,WN,N440LV,1674,14107,PHX,...,LAX,"Los Angeles, CA",2255,2315,N,80,1,370,13.024975,13.024975
3,2020-01-01 00:00:00,WN,WN,WN,3466,WN,N8571Z,3466,14107,PHX,...,LAX,"Los Angeles, CA",1705,1735,N,90,1,370,-0.513752,-0.513752
4,2020-01-01 00:00:00,WN,WN,WN,4657,WN,N7731A,4657,14107,PHX,...,LAX,"Los Angeles, CA",1335,1405,N,90,1,370,-4.727969,-4.727969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20074,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5981,OO,N788SK,5981,10372,ASE,...,LAX,"Los Angeles, CA",740,929,N,169,1,737,-2.018487,-2.018487
20075,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5989,OO,N905SW,5989,13476,MRY,...,LAX,"Los Angeles, CA",1331,1500,N,89,1,266,2.137516,2.137516
20076,2020-01-31 00:00:00,UA,UA_CODESHARE,UA,5994,OO,N167SY,5994,12441,JAC,...,LAX,"Los Angeles, CA",1927,2112,N,165,1,784,-9.122553,-9.122553
20077,2020-01-31 00:00:00,AA,AA_CODESHARE,AA,3239,OO,N703SK,3239,11638,FAT,...,LAX,"Los Angeles, CA",627,743,N,76,1,209,-5.616225,-5.616225


In [137]:
#df_all1=df_all[['fl_date', 'mkt_carrier','mkt_carrier_fl_num','origin','dest','predicted_delay']].copy()

In [138]:
#df_all1['fl_date'][df_all1.fl_date<'2020-01-07 0:00:00'].value_counts()

In [140]:
df_all[['fl_date', 'mkt_carrier','mkt_carrier_fl_num','origin','dest','predicted_delay']][df_all.fl_date<'2020-01-07 0:00:00'].to_csv('sample_submission_LAX.csv',index=False)

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.